In [2]:
import time
from design import *
import importlib
import shutil
from utils import *
from openai import OpenAI
from prompts import *
import json
import numpy as np
from gymnasium.envs.robodesign.GPTHopper import GPTHopperEnv

In [14]:

folder_name = "results/Eureka_morphology"
log_file = os.path.join(folder_name, "parameters.log")
logging.basicConfig(filename=log_file, level=logging.INFO, format="%(asctime)s - %(message)s")

# folder_name = setup_logging(div_flag=True)

best_fitness = float('-inf')  
best_morphology = None  
best_rewardfunc = None  
best_reward = None
best_material = None
best_efficiency = None

iterations = 5
morphology_nums = 16
rewardfunc_nums = 1

fitness_matrix = np.array([[None for _ in range(morphology_nums)] for _ in range(rewardfunc_nums)])
efficiency_matrix = np.array([[None for _ in range(morphology_nums)] for _ in range(rewardfunc_nums)])
fitness_list = []


In [ ]:
import prompts
class DGA:
    def __init__(self):
        self.client = OpenAI(api_key=api_key)
        # self.model = "gpt-3.5-turbo"
        self.model = "gpt-4-turbo"



    def generate_morphology_eureka(self, morphology_nums, best_message, folder_name):

        messages = [
                    {"role": "system", "content": "You are a helpful mujoco robot designer"},
                    {"role": "user", "content": morphology_prompts + best_message + morphology_format}
                    ]
        
        responses = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            response_format={'type': 'json_object'},
            n=morphology_nums
        )

        for i, choice in enumerate(responses.choices):
            print(f"Response {i}:")
            print(json.dumps(choice.message.content, indent=4))

        parameter_list = [json.loads(choice.message.content).get('parameters', []) for choice in responses.choices]
        material_list = [compute_hopper_volume(parameter) for parameter in parameter_list]

        xml_files = []
        for i, parameter in enumerate(parameter_list):
            if not isinstance(parameter, list):
                print(f"Skipping invalid parameter {i}: {parameter}")
                continue

            xml_file = hopper_design(parameter)  
            filename = f"GPTHopper_{i}.xml"
            file_path = os.path.join(folder_name, "assets", filename)
            xml_files.append(file_path)
            with open(file_path, "w") as fp:
                fp.write(xml_file)
            print(f"Successfully saved {filename}")
            
        return xml_files, material_list, parameter_list
    
        


In [16]:
eureka_morphology_prompts = """Role: You are a robot designer trying to design robot parameters to increase the fitness function as effective as possible.
Task: Your task is to design parameters of robot that will help agent achieve the fitness function as high as possible.
fintess function: walk distance/material cost.
Description: The hopper is a two-dimensional one-legged figure consisting of four main body parts - the torso at the top, the thigh in the middle, the leg at the bottom, and a single foot on which the entire body rests.
The goal is to make hops that move in the forward (right) direction by applying torque to the three hinges that connect the four body parts, you should write a reward function to make the robot move as faster as possible.

  <mujoco model="hopper">
    <compiler angle="degree" coordinate="global" inertiafromgeom="true"/>
    <default>
      <joint armature="1" damping="1" limited="true"/>
      <geom conaffinity="1" condim="1" contype="1" margin="0.001" material="geom" rgba="0.8 0.6 .4 1" solimp=".8 .8 .01" solref=".02 1"/>
      <motor ctrllimited="true" ctrlrange="-.4 .4"/>
    </default>
    <option integrator="RK4" timestep="0.002"/>

    <worldbody>
      <light cutoff="100" diffuse="1 1 1" dir="-0 0 -1.3" directional="true" exponent="1" pos="0 0 1.3" specular=".1 .1 .1"/>
      <geom conaffinity="1" condim="3" name="floor" pos="0 0 0" rgba="0.8 0.9 0.8 1" size="20 20 .125" type="plane" material="MatPlane"/>
      <body name="torso">
        <camera name="track" mode="trackcom" pos="0 -3 -0.25" xyaxes="1 0 0 0 0 1"/>
        <joint armature="0" axis="1 0 0" damping="0" limited="false" name="ignore1" pos="0 0 0" stiffness="0" type="slide"/>
        <joint armature="0" axis="0 0 1" damping="0" limited="false" name="ignore2" pos="0 0 0" ref="1.25" stiffness="0" type="slide"/>
        <joint armature="0" axis="0 1 0" damping="0" limited="false" name="ignore3" pos="0 0 0" stiffness="0" type="hinge"/>
        <geom fromto="0 0 {param1} 0 0 {param2}" name="torso_geom" size="{param8}" type="capsule" friction="0.9"/>
        <body name="thigh">
          <joint axis="0 -1 0" name="thigh_joint" pos="0 0 {param2}" range="-150 0" type="hinge"/>
          <geom fromto="0 0 {param2} 0 0 {param3}" name="thigh_geom" size="{param8}" type="capsule" friction="0.9"/>
          <body name="leg">
            <joint axis="0 -1 0" name="leg_joint" pos="0 0 {param3}" range="-150 0" type="hinge"/>
            <geom fromto="0 0 {param3} 0 0 {param4}" name="leg_geom" size="{param9}" type="capsule" friction="0.9"/>
            <body name="foot">
              <joint axis="0 -1 0" name="foot_joint" pos="0 0 {4}" range="-45 45" type="hinge"/>
              <geom fromto="{param5} 0 {param4} {param6} 0 {param4}" name="foot_geom" size="{param10}" type="capsule" friction="2.0"/>
            </body>
          </body>
        </body>
      </body>
    </worldbody>
    <actuator>
      <motor ctrllimited="true" ctrlrange="-1.0 1.0" gear="200.0" joint="thigh_joint"/>
      <motor ctrllimited="true" ctrlrange="-1.0 1.0" gear="200.0" joint="leg_joint"/>
      <motor ctrllimited="true" ctrlrange="-1.0 1.0" gear="200.0" joint="foot_joint"/>
    </actuator>
      <asset>
        <texture type="skybox" builtin="gradient" rgb1=".4 .5 .6" rgb2="0 0 0"
            width="100" height="100"/>
        <texture builtin="flat" height="1278" mark="cross" markrgb="1 1 1" name="texgeom" random="0.01" rgb1="0.8 0.6 0.4" rgb2="0.8 0.6 0.4" type="cube" width="127"/>
        <texture builtin="checker" height="100" name="texplane" rgb1="0 0 0" rgb2="0.8 0.8 0.8" type="2d" width="100"/>
        <material name="MatPlane" reflectance="0.5" shininess="1" specular="1" texrepeat="60 60" texture="texplane"/>
        <material name="geom" texture="texgeom" texuniform="true"/>
      </asset>
  </mujoco>

1. For reducing material cost to ensure the effieciency of robot design, you should reduce redundant paramters (e.g., smaller geom size) and increase paramters who control the robot (e.g., longer leags, ankles).
2. Your design should fit the control gear and others parts of robots well.

There are also some design parameters and their fitness. Please carefully observe these parameters and their fitness, try to design a new parameter to further improve the fitness.

Attention:
1.You must observe the parameters from the highest fitness function and further encourage this design. Also, analyze the low fitness parameters, identify their shortcomings, and avoid those design. 
2.You must carefully observe these parameters and their fitness, try to design a new parameter which is different with the best parameter to further improve the fitness. This is very important.
"""

In [17]:
designer = DGA()
rewardfunc_list = [f'results/Eureka_morphology/env/GPTrewardfunc_{i}.py' for i in range(0,1)]
best_message = ""
for iter in range(iterations):
    morphology_list, material_list, parameter_list = designer.generate_morphology_eureka(morphology_nums, best_message, folder_name)
    for i, rewardfunc in enumerate(rewardfunc_list):
        for j, morphology in enumerate(morphology_list):

            print(i, rewardfunc)
            print(j, morphology)
            shutil.copy(morphology, "GPTHopper.xml")
            shutil.copy(rewardfunc, "GPTrewardfunc.py")         

            import GPTrewardfunc
            importlib.reload(GPTrewardfunc)  # 重新加载模块
            from GPTrewardfunc import _get_rew
            GPTHopperEnv._get_rew = _get_rew

            env_name = "GPTHopperEnv"
            model_path = Train(j,  i, folder_name, total_timesteps=5e5)
            fitness, reward = Eva(model_path)

            material = material_list[j]
            efficiency = fitness/material
            fitness_matrix[i][j] = fitness
            efficiency_matrix[i][j] = efficiency
            
            logging.info("___________________finish coarse optimization_____________________")
            logging.info(f"morphology: {j}, rewardfunc: {i}, material cost: {material} reward: {reward} fitness: {fitness} efficiency: {efficiency}")

    best_index = np.argmax(efficiency_matrix)
    best_efficiency = np.max(efficiency_matrix[:][0])
    best_parameter = parameter_list[best_index]
    best_message = eureka_morphology_prompts + f"\n best parameter:{best_parameter} \n" + f"best fintess:{best_efficiency}" 

Response 0:
"{\n  \"parameters\": [1.4, 1.1, 0.7, 0.3, 0.1, -0.2, 0.03, 0.03, 0.025, 0.02],\n  \"description\": \"This design optimizes the hopper for maximum walking distance per material cost by focusing on efficiency and propulsive capability. Increased torso height creates a high center of gravity aiding forward propulsion. The limbs are sculpted for length, enhancing stride, while minimizing size for reduced material costs. Foot attachments are designed to provide maximum stability and traction during motion.\"\n}"
Response 1:
"{\n  \"parameters\": [1.0, 0.9, 0.55, 0.2, 0.25, -0.25, 0.05, 0.05, 0.04, 0.03],\n  \"desciption\": \"Optimized lightweight design maximizing leg length for effective movement while minimizing body part sizes to reduce material costs.\"\n}"
Response 2:
"{\n  \"parameters\": [1.2, 0.9, 0.6, 0.3, 0.1, -0.05, 0.1, 0.08, 0.07, 0.05],\n  \"description\": \"Optimized model for low material cost and enhanced locomotion efficiency. The torso to foot attachment poin

/root/miniconda3/envs/robodesign/lib/python3.8/site-packages/stable_baselines3/common/save_util.py:284: UserWarning: Path 'results/Eureka_morphology/coarse' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0 results/Eureka_morphology/env/GPTrewardfunc_0.py
1 results/Eureka_morphology/assets/GPTHopper_1.xml
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
0 results/Eureka_morphology/env/GPTrewardfunc_0.py
2 results/Eureka_morphology/assets/GPTHopper_2.xml
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
7

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [21]:
best_efficiency = 998.5637743948149
best_index = 6
best_parameter = [0.7, 0.42, 0.22, 0.09, 0.09, -0.05, 0.04, 0.035, 0.025, 0.015]